In [ ]:
import os
import sys
import lancedb
import ollama
import numpy as np
sys.path.append('../../..')
from backend.LLM.AnythingLLM_client import AnythingLLMClient


dataset_path_txt = "../../../dataset/graduate_txt/faq_txt"
EMBEDDING_MODELS = ["all-minilm:l6-v2", "nomic-embed-text:latest"]
WORKSPACE_NAME = "demo"
embedding_model = EMBEDDING_MODELS[1]
db = lancedb.connect("../../../anythingllm/lancedb")



def initialize_workspace():
    anyllm_client = AnythingLLMClient("http://localhost:3001/api", "YKH2935-TD9MYVY-PDCDN6H-5Z1T0YW")
    anyllm_client.ping_alive()
    return anyllm_client

    
def set_new_parameters(anyllm_client: AnythingLLMClient, embedding_model, max_embedding_size, chunk_size, chunk_overlap):
    response = anyllm_client.set_embedding_provider(embedding_model, max_embedding_size)
    response = anyllm_client.set_chunk_size(chunk_size, chunk_overlap)
    return response
    


def add_files_to_workspace(anyllm_client: AnythingLLMClient, workspace_name, dataset_path):
    files = os.listdir(dataset_path)
    for file in files:
        if file.endswith(".txt"):
            full_path = os.path.join(dataset_path, file)
            anyllm_client.add_document_to_workspace(workspace_name, full_path)
            print(f"Adding file {file} to workspace {workspace_name}")
        


# print(tbl.head())
anyllm_client = initialize_workspace()
add_files_to_workspace(anyllm_client, WORKSPACE_NAME, dataset_path_txt)



In [9]:
tbl = db.open_table("demo")

embed_query = ollama.embeddings(
  model=embedding_model,
  prompt='What is UCL',
)

# print(embed_query.embedding)

df = tbl.search(embed_query.embedding) \
    .limit(4) \
    .to_df()

df

/var/folders/cg/bpchphfn45l7z59lyhbgtm080000gn/T/ipykernel_13433/1020019935.py:10: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  df = tbl.search(embed_query.embedding) \


,id,url,title,docAuthor,description,docSource,chunkSource,published,wordCount,token_count_estimate,text,vector,_distance
0,8b727b48-53c1-48ee-9209-65751d405228,file:///app/collector/hotdir/faq_ucl_question_...,faq_ucl_question_11.txt,Unknown,Unknown,a text file uploaded by the user.,None,"11/23/2024, 1:43:02 PM",209.0,262.0,<document_metadata>\nsourceDocument: faq_ucl_q...,"[0.7314003, -0.11020838, -2.9399054, -0.610837...",346.303314
1,eb05ab4e-c2df-4067-80f9-9dadc038a69a,file:///app/collector/hotdir/faq_ucl_question_...,faq_ucl_question_21.txt,Unknown,Unknown,a text file uploaded by the user.,None,"11/23/2024, 1:43:01 PM",153.0,196.0,<document_metadata>\nsourceDocument: faq_ucl_q...,"[0.85074145, 0.514358, -3.2649925, -0.29782644...",348.383148
2,f3a771da-e150-4656-9e2b-135b03d5d9b3,file:///app/collector/hotdir/faq_ucl_question_...,faq_ucl_question_12.txt,Unknown,Unknown,a text file uploaded by the user.,None,"11/23/2024, 1:43:02 PM",274.0,324.0,<document_metadata>\nsourceDocument: faq_ucl_q...,"[1.1571797, 0.53227746, -2.4045007, -0.4725271...",353.739349
3,0fb48861-4066-4b7d-a4e3-558ff30e7ad6,file:///app/collector/hotdir/faq_ucl_question_...,faq_ucl_question_12.txt,Unknown,Unknown,a text file uploaded by the user.,None,"11/23/2024, 1:43:02 PM",274.0,324.0,<document_metadata>\nsourceDocument: faq_ucl_q...,"[0.8717457, 0.052880906, -2.4759135, -0.654570...",384.379059
